# Modelling lithium plating in PyBaMM

This notebook shows how PyBaMM [7] can be used to model both reversible and irreversible lithium plating.

In [ ]:
%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm
import os
import numpy as np
import matplotlib.pyplot as plt
os.chdir(pybamm.__path__[0]+'/..')

The Doyle-Fuller-Newman model [3] is upgraded with three different lithium plating models. Model 1 contains the reversible lithium plating model of O'Kane et al. [6]. Model 2 contains the same model but with the lithium stripping capability removed, making the plating irreversible. Model 3 contains the updated partially reversible plating of O'Kane et al. [7]. The parameters are taken from Chen et al.'s investigation [2] of an LG M50 cell.

In [ ]:
# choose models
plating_options = ["reversible", "irreversible", "partially reversible"]
models = {option: pybamm.lithium_ion.DFN(options={"lithium plating": option}, name=option) 
          for option in plating_options}

# pick parameters
parameter_values = pybamm.ParameterValues("OKane2022")
parameter_values.update({"Ambient temperature [K]": 268.15})
parameter_values.update({"Upper voltage cut-off [V]": 4.21})
#parameter_values.update({"Lithium plating kinetic rate constant [m.s-1]": 1E-9})
parameter_values.update({"Lithium plating transfer coefficient": 0.5})
parameter_values.update({"Dead lithium decay constant [s-1]": 1E-4})

A series of simple fast charging experiments based on those of Ren et al. [8] is defined here. We first initialise the model at 0% SoC by performing a C/20 discharge (see more details on how to initialise a model from a simulation in [this notebook](./initialise-model-with-solution.ipynb)).

In [ ]:
# specify experiments
pybamm.citations.register("Ren2018")

experiment_discharge = pybamm.Experiment(
    [
        ("Discharge at C/20 until 2.5 V (10 minute period)",
        "Rest for 1 hour (3 minute period)")
    ]
)

sims_discharge = []
for model in models.values():
    sim_discharge = pybamm.Simulation(model, parameter_values=parameter_values, experiment=experiment_discharge)
    sol_discharge = sim_discharge.solve(calc_esoh=False)
    model.set_initial_conditions_from(sol_discharge, inplace=True)
    sims_discharge.append(sim_discharge)

And we can now define the different experiments to charge at different C-rates.

In [ ]:
C_rates = ["2C", "1C", "C/2", "C/4", "C/8"]
experiments = {}
for C_rate in C_rates:
    experiments[C_rate] = pybamm.Experiment(
    [
        (f"Charge at {C_rate} until 4.2 V",
        "Hold at 4.2 V until C/20",
        "Rest for 1 hour")
    ]
)

Solve the reversible plating model first. The default CasADi [1] solver is used here.

In [ ]:
def define_and_solve_sims(model, experiments, parameter_values):
    sims = {}
    for C_rate, experiment in experiments.items():
        sim = pybamm.Simulation(model, experiment=experiment, parameter_values=parameter_values)
        sim.solve(calc_esoh=False)
        sims[C_rate] = sim
        
    return sims

sims_reversible = define_and_solve_sims(models["reversible"], experiments, parameter_values)

In [ ]:
colors = ["tab:purple", "tab:cyan", "tab:red", "tab:green", "tab:blue"]
linestyles = ["dashed", "dotted", "solid"]

param = models["reversible"].param
A = parameter_values.evaluate(param.L_y * param.L_z)
F = parameter_values.evaluate(param.F)
L_n = parameter_values.evaluate(param.n.L)

currents = [
    "X-averaged negative electrode interfacial current density",
    "X-averaged lithium plating interfacial current density",
    "Sum of x-averaged negative electrode volumetric interfacial current densities"
]

def plot(sims):
    fig, axs = plt.subplots(2, 2, figsize=(13,9))
    for (C_rate,sim), color in zip(sims.items(),colors):
        # Isolate final equilibration phase
        sol = sim.solution.cycles[0].steps[2]

        # Voltage vs time
        t = sol["Time [min]"].entries
        t = t-t[0]
        V = sol["Terminal voltage [V]"].entries
        axs[0,0].plot(t, V, color=color, linestyle="solid", label=C_rate)

        # Currents
        for current, ls in zip(currents,linestyles):
            j = sol[current].entries
            axs[0,1].plot(t, j, color=color, linestyle=ls)
        
        # Plated lithium capacity
        Q_Li = sol["Loss of capacity to lithium plating [A.h]"].entries
        axs[1,0].plot(t, Q_Li, color=color, linestyle='solid')
        
        # Capacity vs time
        Q_main = sol["Negative electrode volume-averaged concentration [mol.m-3]"].entries * F * A * L_n / 3600
        axs[1,1].plot(t, Q_main, color=color, linestyle='solid')
        
    axs[0,0].legend()
    axs[0,0].set_ylabel("Voltage [V]")
    axs[0,1].set_ylabel("Interfacial current density [dimensionless]")
    axs[0,1].legend(('Deintercalation current','Stripping current','Total current'))
    axs[1,0].set_ylabel("Plated lithium capacity [Ah]")
    axs[1,1].set_ylabel("Intercalated lithium capacity [Ah]")
    
    for ax in axs.flat:
        ax.set_xlabel("Time [minutes]")

    fig.tight_layout()
    
    return fig, axs

plot(sims_reversible);

The results show both similarities and differences with those of Ren et al. [8]. Notably, unlike Ren et al., this model uses equations [6] that result in a small but finite amount of plated lithium being present in the steady state.

Now solve the irreversible plating model and see how it compares.

In [ ]:
sims_irreversible = define_and_solve_sims(models["irreversible"], experiments, parameter_values)

In [ ]:
plot(sims_irreversible);

Unlike in the reversible case, there is no steady state and the capacity degrades quickly. The lithium inventory decreases by around 40 mAh in just an hour, which is unrealistic. The low temperature fast charge simulations are run one more time, with the partially reversible plating model.

In [ ]:
sims_partially_reversible = define_and_solve_sims(models["partially reversible"], experiments, parameter_values)

In [ ]:
plot(sims_partially_reversible);

The partially reversible plating model has features of both the reversible and irreversible models, which is reflected in the results. The plated lithium capacity decreases with time as lithium is reversibly stripped, but the final plated lithium capacity now depends on charge rate, indicating that some lithium was irreversibly plated during charge.

# References

In [ ]:
pybamm.print_citations()